This script process DNA and RNA  integration results and merge them together side by side. It is written for Cervical project.

In [1]:
import matplotlib
import seaborn
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import scipy.stats as ss
from matplotlib import style
matplotlib.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (15, 9)
import pandas as pd
# figure out number of indels

This function process groupby object, combine all rows with the same patient/virus indexed entries into one row

In [36]:
def process_grp(grp):
#     _df_or_sr = grp.apply(lambda col: ';'.join(col.unique().tolist()))
#     lambda x: True if x % 2 == 0 else False
    _df_or_sr = grp.apply(lambda col: ';'.join(col.unique().tolist()) if len(col.unique())==1 else ';'.join(col.tolist()))
    _df = _df_or_sr.to_frame().T
    assert _df.shape[0] == 1
    return _df

take in concatenated integration results and add in patient ids

In [37]:
f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/124_patients/RNA_integration_results.txt'
rna_itg = pd.read_csv(f1, sep='\t')
rna_itg.head(2)

,library,virus,Integration,human-breakpoint,human-gene(s)
0,A78501,Human papillomavirus type 45,YES,3_195595583,TNK2
1,A78501,Human papillomavirus type 45,YES,3_195595582,TNK2


In [39]:
f2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients/124_patients_bam_vcf.txt'
patients = pd.read_csv(f2, sep='\t', usecols=['patient', 'source', 'RNA_lib', 'status'])
patients = patients.rename(columns = {'RNA_lib':'library'})
patients = patients[patients.status == 'Malignant']
assert patients.shape[0] == 124
print(patients.shape)
patients.head()

(124, 4)


,patient,source,status,library
0,HTMCP-03-06-02001,HTMCP_124,Malignant,A37700
1,HTMCP-03-06-02002,HTMCP_125,Malignant,A37701
2,HTMCP-03-06-02003,HTMCP_126,Malignant,A37702
3,HTMCP-03-06-02006,HTMCP_129,Malignant,A37704
4,HTMCP-03-06-02007,HTMCP_130,Malignant,A37705


In [40]:
rna_itg = pd.merge(rna_itg, patients, left_on='library', right_on='library', how='outer')
rna_itg.head()

,library,virus,Integration,human-breakpoint,human-gene(s),patient,source,status
0,A78501,Human papillomavirus type 45,YES,3_195595583,TNK2,HTMCP-03-06-02447,HTMCP_719,Malignant
1,A78501,Human papillomavirus type 45,YES,3_195595582,TNK2,HTMCP-03-06-02447,HTMCP_719,Malignant
2,A78501,Human papillomavirus type 45,YES,3_195595581,TNK2,HTMCP-03-06-02447,HTMCP_719,Malignant
3,A78501,Human papillomavirus type 45,YES,3_195595580,TNK2,HTMCP-03-06-02447,HTMCP_719,Malignant
4,A78501,Human papillomavirus type 45,YES,3_195595579,TNK2,HTMCP-03-06-02447,HTMCP_719,Malignant


In [41]:
print(rna_itg.shape)
rna_itg = rna_itg[rna_itg['virus'] != 'No-Hits']
print(rna_itg.shape)
rna_itg.head()

(855, 8)
(847, 8)


,library,virus,Integration,human-breakpoint,human-gene(s),patient,source,status
0,A78501,Human papillomavirus type 45,YES,3_195595583,TNK2,HTMCP-03-06-02447,HTMCP_719,Malignant
1,A78501,Human papillomavirus type 45,YES,3_195595582,TNK2,HTMCP-03-06-02447,HTMCP_719,Malignant
2,A78501,Human papillomavirus type 45,YES,3_195595581,TNK2,HTMCP-03-06-02447,HTMCP_719,Malignant
3,A78501,Human papillomavirus type 45,YES,3_195595580,TNK2,HTMCP-03-06-02447,HTMCP_719,Malignant
4,A78501,Human papillomavirus type 45,YES,3_195595579,TNK2,HTMCP-03-06-02447,HTMCP_719,Malignant


In [119]:
RNA_GRP = rna_itg.groupby(['patient', 'virus']).apply(process_grp).reset_index(level=2, drop=True)
del RNA_GRP['patient']
del RNA_GRP['virus']
# RNA_GRP
RNA_GRP.reset_index(inplace=True)

print(rna_itg.head(3))
header_dict = {i:'_'.join(['RNA', i]) for i in RNA_GRP.columns.tolist()}
RNA_GRP.rename(columns=header_dict, inplace=True)
RNA_GRP.head()

  library                         virus Integration human-breakpoint  \
0  A78501  Human papillomavirus type 45         YES      3_195595583   
1  A78501  Human papillomavirus type 45         YES      3_195595582   
2  A78501  Human papillomavirus type 45         YES      3_195595581   

  human-gene(s)            patient     source     status  
0          TNK2  HTMCP-03-06-02447  HTMCP_719  Malignant  
1          TNK2  HTMCP-03-06-02447  HTMCP_719  Malignant  
2          TNK2  HTMCP-03-06-02447  HTMCP_719  Malignant  


,RNA_patient,RNA_virus,RNA_library,RNA_Integration,RNA_human-breakpoint,RNA_human-gene(s),RNA_source,RNA_status
0,HTMCP-03-06-02001,Human papillomavirus type 16,A37700,YES,3_189542968;3_189542598;3_189542597;3_18952631...,TP63,HTMCP_124,Malignant
1,HTMCP-03-06-02002,Human papillomavirus type 52,A37701,YES,22_43168251;22_43165135;22_43163674;22_4316152...,intergenic,HTMCP_125,Malignant
2,HTMCP-03-06-02003,Human papillomavirus type 16,A37702,YES,11_60935033;11_60882522,intergenic;CD5,HTMCP_126,Malignant
3,HTMCP-03-06-02006,Human papillomavirus type 18,A37704,NO,-,-,HTMCP_129,Malignant
4,HTMCP-03-06-02006,Human papillomavirus type 45,A37704,YES,12_46339702;12_46337036;12_46337035;12_4632994...,SCAF11;SCAF11;SCAF11;SCAF11;SCAF11;ARID2,HTMCP_129,Malignant


In [120]:
# dna integration results
f3 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/124_patients/DNA_integration_results.txt'
dna_itg = pd.read_csv(f3, sep='\t')
dna_itg.head(2)

,library,virus,Integration,human-breakpoint,human-gene(s)
0,HTMCP_719,No-Hits,NO,-,-
1,HTMCP_716,Human papillomavirus type 18,YES,X_97127953,intergenic


In [121]:
pat_tmp = patients.drop('library', axis=1)

In [122]:
dna_itg = pd.merge(dna_itg, pat_tmp, left_on='library', right_on='source', how='outer')
dna_itg.head(2)

,library,virus,Integration,human-breakpoint,human-gene(s),patient,source,status
0,HTMCP_719,No-Hits,NO,-,-,HTMCP-03-06-02447,HTMCP_719,Malignant
1,HTMCP_716,Human papillomavirus type 18,YES,X_97127953,intergenic,HTMCP-03-06-02442,HTMCP_716,Malignant


In [123]:
print(dna_itg.shape)
dna_itg = dna_itg[dna_itg['virus'] != 'No-Hits']
print(dna_itg.shape)

(426, 8)
(365, 8)


In [124]:
DNA_GRP = dna_itg.groupby(['patient', 'virus']).apply(process_grp).reset_index(level=2, drop=True)
del DNA_GRP['patient']
del DNA_GRP['virus']
# DNA_GRP
DNA_GRP.reset_index(inplace=True)

print(dna_itg.head(3))
DNA_GRP.head()
header_dict = {i:'_'.join(['DNA', i]) for i in DNA_GRP.columns.tolist()}
DNA_GRP.rename(columns=header_dict, inplace=True)
DNA_GRP.head()

     library                         virus Integration human-breakpoint  \
1  HTMCP_716  Human papillomavirus type 18         YES       X_97127953   
2  HTMCP_716  Human papillomavirus type 18         YES      13_74004235   
3  HTMCP_716  Human papillomavirus type 18         YES      13_73987106   

  human-gene(s)            patient     source     status  
1    intergenic  HTMCP-03-06-02442  HTMCP_716  Malignant  
2    intergenic  HTMCP-03-06-02442  HTMCP_716  Malignant  
3    intergenic  HTMCP-03-06-02442  HTMCP_716  Malignant  


,DNA_patient,DNA_virus,DNA_library,DNA_Integration,DNA_human-breakpoint,DNA_human-gene(s),DNA_source,DNA_status
0,HTMCP-03-06-02003,Human papillomavirus type 16,HTMCP_126,NO,-,-,HTMCP_126,Malignant
1,HTMCP-03-06-02012,Human papillomavirus type 34,HTMCP_133,NO,-,-,HTMCP_133,Malignant
2,HTMCP-03-06-02012,Human papillomavirus type 73,HTMCP_133,YES,17_36310359,intergenic,HTMCP_133,Malignant
3,HTMCP-03-06-02036,Human papillomavirus type 16,HTMCP_142,NO,-,-,HTMCP_142,Malignant
4,HTMCP-03-06-02037,Human papillomavirus type 52,HTMCP_143,NO,-,-,HTMCP_143,Malignant


In [125]:
# rename patient and virus to common names for outer merge. 
# not doing this step results in 2 columns for each data set, unique entries for each set will be all na
# itg_merge = pd.merge(DNA_GRP, RNA_GRP, left_on=['DNA_patient', 'DNA_virus'], how='outer',right_on=['RNA_patient', 'RNA_virus'])
DNA_GRP = DNA_GRP.rename(columns={'DNA_patient': 'patient', 'DNA_virus':'virus', 'DNA_source':'source_id'})
RNA_GRP = RNA_GRP.rename(columns={'RNA_patient': 'patient', 'RNA_virus':'virus', 'RNA_source':'source_id'})
itg_merge = pd.merge(DNA_GRP, RNA_GRP, on=['patient', 'source_id', 'virus'], how='outer')

In [126]:
itg_merge.fillna('na', inplace=True)
itg_merge.head(2)

,patient,virus,DNA_library,DNA_Integration,DNA_human-breakpoint,DNA_human-gene(s),source_id,DNA_status,RNA_library,RNA_Integration,RNA_human-breakpoint,RNA_human-gene(s),RNA_status
0,HTMCP-03-06-02003,Human papillomavirus type 16,HTMCP_126,NO,-,-,HTMCP_126,Malignant,A37702,YES,11_60935033;11_60882522,intergenic;CD5,Malignant
1,HTMCP-03-06-02012,Human papillomavirus type 34,HTMCP_133,NO,-,-,HTMCP_133,Malignant,A37707,NO,-,-,Malignant


In [127]:
# remap dna and rna ids since some of them are na due to merge
df7 = pd.merge(itg_merge, patients, left_on='patient', right_on='patient', how='outer')
df7.head()

,patient,virus,DNA_library,DNA_Integration,DNA_human-breakpoint,DNA_human-gene(s),source_id,DNA_status,RNA_library,RNA_Integration,RNA_human-breakpoint,RNA_human-gene(s),RNA_status,source,status,library
0,HTMCP-03-06-02003,Human papillomavirus type 16,HTMCP_126,NO,-,-,HTMCP_126,Malignant,A37702,YES,11_60935033;11_60882522,intergenic;CD5,Malignant,HTMCP_126,Malignant,A37702
1,HTMCP-03-06-02012,Human papillomavirus type 34,HTMCP_133,NO,-,-,HTMCP_133,Malignant,A37707,NO,-,-,Malignant,HTMCP_133,Malignant,A37707
2,HTMCP-03-06-02012,Human papillomavirus type 73,HTMCP_133,YES,17_36310359,intergenic,HTMCP_133,Malignant,A37707,YES,6_92985326;17_36401125;17_36347084;17_36285541,"intergenic;LOC440434;TBC1D3,LOC440434;TBC1D3",Malignant,HTMCP_133,Malignant,A37707
3,HTMCP-03-06-02012,Human papillomavirus type 16,na,na,na,na,HTMCP_133,na,A37707,YES,14_38057621;14_38057528,intergenic,Malignant,HTMCP_133,Malignant,A37707
4,HTMCP-03-06-02012,Human papillomavirus type 18,na,na,na,na,HTMCP_133,na,A37707,NO,-,-,Malignant,HTMCP_133,Malignant,A37707


In [128]:
of = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/124_patients/124_patients_integration_details_merged.txt'
df7.to_csv(of, index=False)


In [92]:
df = itg_merge[['patient', 'virus', 'DNA_Integration', 'RNA_Integration']]

In [129]:
# df.set_index(['patient', 'virus'])

In [131]:
# 106 patient integration results
f6 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/107_patients/106_integration.txt'
df6 = pd.read_csv(f6)
df6.head()

,virus,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02007,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,...,HTMCP-03-06-02328,HTMCP-03-06-02330,HTMCP-03-06-02332,HTMCP-03-06-02333,HTMCP-03-06-02335,HTMCP-03-06-02340,HTMCP-03-06-02341,HTMCP-03-06-02344,HTMCP-03-06-02346,HTMCP-03-06-02354
0,HPV_16,integrated,NaN,integrated,NaN,NaN,unintegrated,integrated,integrated,NaN,...,NaN,NaN,integrated,integrated,NaN,NaN,NaN,integrated,NaN,NaN
1,HPV_18,NaN,NaN,NaN,NaN,NaN,NaN,unintegrated,NaN,NaN,...,NaN,NaN,NaN,NaN,unintegrated,NaN,integrated,NaN,unintegrated,integrated
2,HPV_26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HPV_30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,integrated,NaN,NaN,NaN,NaN
4,HPV_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,unintegrated,NaN,NaN,NaN


In [149]:
df8 = df6.set_index('virus').T.drop(['HIV_status', 'Putative_histology'], axis=1).stack().reset_index()
df8.columns = names=['patient', 'virus', '107_results']
df8.head()

,patient,virus,107_results
0,HTMCP-03-06-02001,HPV_16,integrated
1,HTMCP-03-06-02002,HPV_52,integrated
2,HTMCP-03-06-02003,HPV_16,integrated
3,HTMCP-03-06-02006,HPV_45,integrated
4,HTMCP-03-06-02008,HPV_16,unintegrated


In [155]:
# 124 patient integration results
f9 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/124_patients/124_patients_integration_details_merged_reviewed.txt'
df9 = pd.read_csv(f9)
df9.head()

,patient,virus,integration_reported,integration_detected,source,DNA_Integration,DNA_human-breakpoint,DNA_human-gene(s),library,RNA_Integration,RNA_human-breakpoint,RNA_human-gene(s)
0,HTMCP-03-06-02012,HPV_73,integrated,BOTH_INTEGRATED,HTMCP_133,YES,17_36310359,intergenic,A37707,YES,6_92985326;17_36401125;17_36347084;17_36285541,"intergenic;LOC440434;TBC1D3,LOC440434;TBC1D3"
1,HTMCP-03-06-02042,HPV_16,integrated,BOTH_INTEGRATED,HTMCP_145,YES,1_151029496;1_151029494,CDC42SE1,A37717,YES,1_151039697;1_151039693;1_151032954;1_15103285...,MLLT11;MLLT11;MLLT11;MLLT11;CDC42SE1;CDC42SE1;...
2,HTMCP-03-06-02046,HPV_16,integrated,BOTH_INTEGRATED,HTMCP_147,YES,19_1192727;19_1183502;19_1181971,intergenic,A37718,YES,19_1227589;19_1227585;19_1227517;19_1227505;19...,STK11;STK11;STK11;STK11;STK11;intergenic;inter...
3,HTMCP-03-06-02058,HPV_18,integrated,BOTH_INTEGRATED,HTMCP_151,YES,8_128323246;8_128319365;18_37408866,"DQ515899,DQ515898,LOC727677;DQ515899,DQ515898,...",A37722,YES,8_128323246;8_128321637;8_128319365;8_12831205...,"DQ515899,DQ515898,LOC727677;DQ515899,DQ515898,..."
4,HTMCP-03-06-02063,HPV_16,integrated,BOTH_INTEGRATED,HTMCP_156,YES,22_42896541;2_142008771;2_141933994,intergenic;LRP1B;LRP1B,A37725,YES,22_42950051;22_42950048;2_142010270;2_14201026...,SERHL2;SERHL2;LRP1B;LRP1B;LRP1B;LRP1B;LRP1B


In [156]:
df10 = pd.merge(df9, df8, how='left', left_on=['patient', 'virus'], right_on=['patient', 'virus'])

In [153]:
df10.head()

,patient,virus,concensus,source,DNA_Integration,DNA_human-breakpoint,DNA_human-gene(s),library,RNA_Integration,RNA_human-breakpoint,RNA_human-gene(s),107_results
0,HTMCP-03-06-02001,HPV_16,RNA_INTEGRATED,HTMCP_124,na,na,na,A37700,YES,3_189542968;3_189542598;3_189542597;3_18952631...,TP63,integrated
1,HTMCP-03-06-02002,HPV_52,RNA_INTEGRATED,HTMCP_125,na,na,na,A37701,YES,22_43168251;22_43165135;22_43163674;22_4316152...,intergenic,integrated
2,HTMCP-03-06-02003,HPV_16,RNA_INTEGRATED,HTMCP_126,NO,-,-,A37702,YES,11_60935033;11_60882522,intergenic;CD5,integrated
3,HTMCP-03-06-02006,HPV_18,NOT_INTEGRATED,HTMCP_129,na,na,na,A37704,NO,-,-,NaN
4,HTMCP-03-06-02006,HPV_45,RNA_INTEGRATED,HTMCP_129,na,na,na,A37704,YES,12_46339702;12_46337036;12_46337035;12_4632994...,SCAF11;SCAF11;SCAF11;SCAF11;SCAF11;ARID2,integrated


In [174]:
# results of 101 patients
f11 = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/101_patients/reviewed_integration_results_101_patients.csv'
df11 = pd.read_csv(f11)
df11.head()

,ids,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02007,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,...,HTMCP-03-06-02266,HTMCP-03-06-02267,HTMCP-03-06-02268,HTMCP-03-06-02320,HTMCP-03-06-02322,HTMCP-03-06-02326,HTMCP-03-06-02328,HTMCP-03-06-02330,HTMCP-03-06-02332,HTMCP-03-06-02333
0,HPV_16,integrated,NaN,integrated,NaN,NaN,unintegrated,integrated,integrated,NaN,...,NaN,NaN,NaN,NaN,integrated,unintegrated,NaN,NaN,integrated,integrated
1,HPV_18,NaN,NaN,NaN,NaN,NaN,NaN,unintegrated,NaN,NaN,...,NaN,integrated,unintegrated,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HPV_26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HPV_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HPV_33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
df11

,ids,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02007,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,...,HTMCP-03-06-02266,HTMCP-03-06-02267,HTMCP-03-06-02268,HTMCP-03-06-02320,HTMCP-03-06-02322,HTMCP-03-06-02326,HTMCP-03-06-02328,HTMCP-03-06-02330,HTMCP-03-06-02332,HTMCP-03-06-02333
0,HPV_16,integrated,NaN,integrated,NaN,NaN,unintegrated,integrated,integrated,NaN,...,NaN,NaN,NaN,NaN,integrated,unintegrated,NaN,NaN,integrated,integrated
1,HPV_18,NaN,NaN,NaN,NaN,NaN,NaN,unintegrated,NaN,NaN,...,NaN,integrated,unintegrated,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HPV_26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HPV_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HPV_33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,HPV_34,NaN,NaN,NaN,NaN,NaN,NaN,unintegrated,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,HPV_35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,unintegrated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,HPV_39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,unintegrated,NaN,NaN,NaN,unintegrated,NaN,NaN,NaN
8,HPV_45,NaN,NaN,NaN,integrated,NaN,NaN,NaN,NaN,integrated,...,integrated,NaN,unintegrated,integrated,NaN,integrated,unintegrated,NaN,NaN,NaN
9,HPV_51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
df11 = df11.set_index('ids').T.drop(['HIV_status', 'putative_Histology'], axis=1).stack().reset_index()
df11.columns = names=['patient', 'virus', '101_results']
df11.head()

,patient,virus,101_results
0,HTMCP-03-06-02001,HPV_16,integrated
1,HTMCP-03-06-02002,HPV_52,integrated
2,HTMCP-03-06-02003,HPV_16,integrated
3,HTMCP-03-06-02006,HPV_45,integrated
4,HTMCP-03-06-02008,HPV_16,unintegrated


In [177]:
df12 = pd.merge(df10, df11, how='left', left_on=['patient', 'virus'], right_on=['patient', 'virus'])

In [178]:
df12.head()

,patient,virus,integration_reported,integration_detected,source,DNA_Integration,DNA_human-breakpoint,DNA_human-gene(s),library,RNA_Integration,RNA_human-breakpoint,RNA_human-gene(s),107_results,101_results
0,HTMCP-03-06-02012,HPV_73,integrated,BOTH_INTEGRATED,HTMCP_133,YES,17_36310359,intergenic,A37707,YES,6_92985326;17_36401125;17_36347084;17_36285541,"intergenic;LOC440434;TBC1D3,LOC440434;TBC1D3",integrated,integrated
1,HTMCP-03-06-02042,HPV_16,integrated,BOTH_INTEGRATED,HTMCP_145,YES,1_151029496;1_151029494,CDC42SE1,A37717,YES,1_151039697;1_151039693;1_151032954;1_15103285...,MLLT11;MLLT11;MLLT11;MLLT11;CDC42SE1;CDC42SE1;...,integrated,integrated
2,HTMCP-03-06-02046,HPV_16,integrated,BOTH_INTEGRATED,HTMCP_147,YES,19_1192727;19_1183502;19_1181971,intergenic,A37718,YES,19_1227589;19_1227585;19_1227517;19_1227505;19...,STK11;STK11;STK11;STK11;STK11;intergenic;inter...,integrated,integrated
3,HTMCP-03-06-02058,HPV_18,integrated,BOTH_INTEGRATED,HTMCP_151,YES,8_128323246;8_128319365;18_37408866,"DQ515899,DQ515898,LOC727677;DQ515899,DQ515898,...",A37722,YES,8_128323246;8_128321637;8_128319365;8_12831205...,"DQ515899,DQ515898,LOC727677;DQ515899,DQ515898,...",integrated,integrated
4,HTMCP-03-06-02063,HPV_16,integrated,BOTH_INTEGRATED,HTMCP_156,YES,22_42896541;2_142008771;2_141933994,intergenic;LRP1B;LRP1B,A37725,YES,22_42950051;22_42950048;2_142010270;2_14201026...,SERHL2;SERHL2;LRP1B;LRP1B;LRP1B;LRP1B;LRP1B,integrated,integrated


In [184]:
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/124_patients/124_patients_integration_details_merged_comparison.txt'
df12.fillna('not_detected', inplace=True)
df12.to_csv(f, index=False)

# manual review integration results then make oncoprint:

In [2]:
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/124_patients/124_patients_integration_details_merged_comparison.xls'
df = pd.read_excel(f)
df = df
df.head()

,patient,virus,reviewed_results,manual_review_notes,source,DNA_Integration,DNA_human-breakpoint,DNA_human-gene(s),library,RNA_Integration,RNA_human-breakpoint,RNA_human-gene(s),124_results,107_results,101_results
0,HTMCP-03-06-02001,HPV_16,integrated,not_reviewed,HTMCP_124,na,na,na,A37700,YES,3_189542968;3_189542598;3_189542597;3_18952631...,TP63,integrated,integrated,integrated
1,HTMCP-03-06-02002,HPV_52,integrated,not_reviewed,HTMCP_125,na,na,na,A37701,YES,22_43168251;22_43165135;22_43163674;22_4316152...,intergenic,integrated,integrated,integrated
2,HTMCP-03-06-02003,HPV_16,integrated,not_reviewed,HTMCP_126,NO,-,-,A37702,YES,11_60935033;11_60882522,intergenic;CD5,integrated,integrated,integrated
3,HTMCP-03-06-02006,HPV_18,not_detected,reviewed,HTMCP_129,na,na,na,A37704,NO,-,-,NOT_INTEGRATED,not_detected,not_detected
4,HTMCP-03-06-02006,HPV_45,integrated,not_reviewed,HTMCP_129,na,na,na,A37704,YES,12_46339702;12_46337036;12_46337035;12_4632994...,SCAF11;SCAF11;SCAF11;SCAF11;SCAF11;ARID2,integrated,integrated,integrated


In [104]:
df = df[['patient', 'virus', 'reviewed_results']]
df.head()

,patient,virus,reviewed_results
0,HTMCP-03-06-02001,HPV_16,integrated
1,HTMCP-03-06-02002,HPV_52,integrated
2,HTMCP-03-06-02003,HPV_16,integrated
3,HTMCP-03-06-02006,HPV_18,not_detected
4,HTMCP-03-06-02006,HPV_45,integrated


In [105]:
df.shape

(193, 3)

In [106]:
# add in hiv status and histology
# get meta track for example histology
f3 ='/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients/mutation_load_clinic.txt'
df3 = pd.read_csv(f3, sep='\t')
df3 = df3[['patient', 'reanne_HIV_status', 'reanne_Putative_histology']]
df3 = df3.set_index('patient').stack().reset_index()
df3.head(2)

,patient,level_1,0
0,HTMCP-03-06-02001,reanne_HIV_status,Positive
1,HTMCP-03-06-02001,reanne_Putative_histology,Squamous


In [114]:
# give the same header:
df3.columns = ['patient', 'virus', 'reviewed_results']

In [115]:
df3.head()

,patient,virus,reviewed_results
0,HTMCP-03-06-02001,reanne_HIV_status,Positive
1,HTMCP-03-06-02001,reanne_Putative_histology,Squamous
2,HTMCP-03-06-02002,reanne_HIV_status,Negative
3,HTMCP-03-06-02002,reanne_Putative_histology,Squamous
4,HTMCP-03-06-02003,reanne_HIV_status,Positive


In [119]:
dfc = pd.concat([df, df3])
dfc.head(2)

,patient,virus,reviewed_results
0,HTMCP-03-06-02001,HPV_16,integrated
1,HTMCP-03-06-02002,HPV_52,integrated


In [123]:
assert dfc.patient.nunique() == 124

In [138]:
dfp = dfc.pivot(index='patient', columns='virus', values='reviewed_results').drop(['APV_9', 'no_HPV'], axis=1).T
# del dfp.index.name
dfp.head()
# dfp.shape

patient,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02007,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02026,...,HTMCP-03-06-02424,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447,HTMCP-03-06-02448
virus,,,,,,,,,,,,,,,,,,,,,
HPV_16,integrated,None,integrated,None,None,unintegrated,integrated,integrated,None,None,...,integrated,None,integrated,None,None,integrated,None,None,None,None
HPV_18,None,None,None,not_detected,None,None,unintegrated,None,not_detected,None,...,None,integrated,None,integrated,None,None,integrated,integrated,None,None
HPV_26,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
HPV_30,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
HPV_31,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [140]:
viruses = dfp.index.tolist()
viruses

['HPV_16',
 'HPV_18',
 'HPV_26',
 'HPV_30',
 'HPV_31',
 'HPV_33',
 'HPV_34',
 'HPV_35',
 'HPV_39',
 'HPV_45',
 'HPV_51',
 'HPV_52',
 'HPV_53',
 'HPV_58',
 'HPV_59',
 'HPV_6',
 'HPV_68',
 'HPV_69',
 'HPV_73',
 'HPV_82',
 'HPV_9',
 'HPV_97',
 'reanne_HIV_status',
 'reanne_Putative_histology']

In [144]:
dfp['occurrence'] = (dfp.notnull().sum(axis=1)/(dfp.shape[1] -2))

In [148]:
viruses = dfp['occurrence'].sort_values(ascending=False).index.tolist()
viruses

['reanne_Putative_histology',
 'reanne_HIV_status',
 'HPV_16',
 'HPV_18',
 'HPV_45',
 'HPV_52',
 'HPV_58',
 'HPV_82',
 'HPV_51',
 'HPV_68',
 'HPV_59',
 'HPV_31',
 'HPV_97',
 'HPV_35',
 'HPV_33',
 'HPV_9',
 'HPV_53',
 'HPV_69',
 'HPV_39',
 'HPV_6',
 'HPV_34',
 'HPV_30',
 'HPV_26',
 'HPV_73']

In [155]:
viruses = [
 'reanne_HIV_status',
    'reanne_Putative_histology',
 'HPV_16',
 'HPV_18',
 'HPV_45',
 'HPV_52',
 'HPV_58',
 'HPV_82',
 'HPV_51',
 'HPV_68',
 'HPV_59',
 'HPV_31',
 'HPV_97',
 'HPV_35',
 'HPV_33',
 'HPV_9',
 'HPV_53',
 'HPV_69',
 'HPV_39',
 'HPV_6',
 'HPV_34',
 'HPV_30',
 'HPV_26',
 'HPV_73']

In [156]:
dfp = dfp.reindex(viruses)
dfp.head()

patient,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02007,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02026,...,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447,HTMCP-03-06-02448,occurrence,percentage
virus,,,,,,,,,,,,,,,,,,,,,
reanne_HIV_status,Positive,Negative,Positive,Negative,Negative,Negative,Negative,Negative,Positive,Negative,...,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,1.008197,HIV_Status
reanne_Putative_histology,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous,Squamous,Squamous,Adeno,...,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous,Adeno,1.008197,Puatative_Histology
HPV_16,integrated,None,integrated,None,None,unintegrated,integrated,integrated,None,None,...,integrated,None,None,integrated,None,None,None,None,0.426230,HPV_16(43%)
HPV_18,None,None,None,not_detected,None,None,unintegrated,None,not_detected,None,...,None,integrated,None,None,integrated,integrated,None,None,0.311475,HPV_18(31%)
HPV_45,None,None,None,integrated,None,None,None,None,integrated,None,...,None,None,None,None,None,None,integrated,None,0.196721,HPV_45(20%)


In [157]:
dfp['percentage'] = ['{0}({1}%)'.format(i[0], int(round(i[1]*100))) for i in zip(dfp.index.tolist(), dfp.occurrence)]
dfp.loc['reanne_HIV_status', 'percentage'] = 'HIV_Status'
dfp.loc['reanne_Putative_histology', 'percentage' ] = 'Puatative_Histology'

In [158]:
dff = dfp.drop('occurrence', axis=1).set_index('percentage', drop=True)

In [159]:
dff.head()

patient,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02007,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02026,...,HTMCP-03-06-02424,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447,HTMCP-03-06-02448
percentage,,,,,,,,,,,,,,,,,,,,,
HIV_Status,Positive,Negative,Positive,Negative,Negative,Negative,Negative,Negative,Positive,Negative,...,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive
Puatative_Histology,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous,Squamous,Squamous,Adeno,...,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous,Adeno
HPV_16(43%),integrated,None,integrated,None,None,unintegrated,integrated,integrated,None,None,...,integrated,None,integrated,None,None,integrated,None,None,None,None
HPV_18(31%),None,None,None,not_detected,None,None,unintegrated,None,not_detected,None,...,None,integrated,None,integrated,None,None,integrated,integrated,None,None
HPV_45(20%),None,None,None,integrated,None,None,None,None,integrated,None,...,None,None,None,None,None,None,None,None,integrated,None


In [58]:
# dfm = df.set_index(['virus', 'patient'])['reviewed_results'].unstack().drop(['APV_9', 'no_HPV'])

In [160]:
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/124_patients/124_patients_integration_details_merged_comparison_oncoprint.txt'
dff.to_csv(f)

In [185]:
# df12[df12['101_results'] != df12['107_results']][['patient', 'virus','101_results', '107_results']].sort_values('patient')

In [162]:
diff = df10[df10.integration_reported != df10['107_results']][['patient', 'virus','integration_reported', '107_results']]

In [163]:
diff.head()

,patient,virus,integration_reported,107_results
14,HTMCP-03-06-02176,HPV_58,integrated,NaN
38,HTMCP-03-06-02392,HPV_33,integrated,NaN
39,HTMCP-03-06-02393,HPV_16,integrated,NaN
40,HTMCP-03-06-02411,HPV_16,integrated,NaN
41,HTMCP-03-06-02427,HPV_18,integrated,NaN


In [165]:
diff[diff.virus != 'APV_9'].sort_values('patient')

,patient,virus,integration_reported,107_results
64,HTMCP-03-06-02006,HPV_18,NOT_INTEGRATED,NaN
59,HTMCP-03-06-02007,na,NOT_DETECTED,NaN
69,HTMCP-03-06-02020,HPV_18,unintegrated,NaN
60,HTMCP-03-06-02026,na,NOT_DETECTED,NaN
72,HTMCP-03-06-02061,HPV_16,unintegrated,NaN
147,HTMCP-03-06-02068,HPV_59,integrated,NaN
61,HTMCP-03-06-02085,na,NOT_DETECTED,NaN
76,HTMCP-03-06-02092,HPV_18,unintegrated,NaN
45,HTMCP-03-06-02098,HPV_9,integrated,NaN
48,HTMCP-03-06-02110,HPV_45,integrated,NaN


In [105]:
patients.head(2)

,patient,source,library
0,HTMCP-03-06-02001,HTMCP_124,A37700
1,HTMCP-03-06-02002,HTMCP_125,A37701


In [106]:
df7 = pd.merge(df6, patients, left_on='patient', right_on='patient', how='outer')

In [107]:
df7.head()

,patient,virus,DNA_library,DNA_Integration,DNA_human-breakpoint,DNA_human-gene(s),source_id,RNA_library,RNA_Integration,RNA_human-breakpoint,RNA_human-gene(s),source,library
0,HTMCP-03-06-02001,HPV_16,na,na,na,na,HTMCP_124,A37700,YES,3_189542968;3_189542598;3_189542597;3_18952631...,TP63,HTMCP_124,A37700
1,HTMCP-03-06-02001,HPV_16,na,na,na,na,HTMCP_124,A37700,YES,3_189542968;3_189542598;3_189542597;3_18952631...,TP63,HTMCP_176,NaN
2,HTMCP-03-06-02002,HPV_52,na,na,na,na,HTMCP_125,A37701,YES,22_43168251;22_43165135;22_43163674;22_4316152...,intergenic,HTMCP_125,A37701
3,HTMCP-03-06-02002,HPV_52,na,na,na,na,HTMCP_125,A37701,YES,22_43168251;22_43165135;22_43163674;22_4316152...,intergenic,HTMCP_177,NaN
4,HTMCP-03-06-02003,HPV_16,HTMCP_126,NO,-,-,HTMCP_126,A37702,YES,11_60935033;11_60882522,intergenic;CD5,HTMCP_126,A37702


# what genes are hit by HPV

In [29]:
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/124_patients/124_patients_integration_details_merged_comparison.xls'
df = pd.read_excel(f)
df.head()

,patient,virus,reviewed_results,manual_review_notes,source,DNA_Integration,DNA_human-breakpoint,DNA_human-gene(s),library,RNA_Integration,RNA_human-breakpoint,RNA_human-gene(s),124_results,107_results,101_results
0,HTMCP-03-06-02001,HPV_16,integrated,not_reviewed,HTMCP_124,na,na,na,A37700,YES,3_189542968;3_189542598;3_189542597;3_18952631...,TP63,integrated,integrated,integrated
1,HTMCP-03-06-02002,HPV_52,integrated,not_reviewed,HTMCP_125,na,na,na,A37701,YES,22_43168251;22_43165135;22_43163674;22_4316152...,intergenic,integrated,integrated,integrated
2,HTMCP-03-06-02003,HPV_16,integrated,not_reviewed,HTMCP_126,NO,-,-,A37702,YES,11_60935033;11_60882522,intergenic;CD5,integrated,integrated,integrated
3,HTMCP-03-06-02006,HPV_18,not_detected,reviewed,HTMCP_129,na,na,na,A37704,NO,-,-,NOT_INTEGRATED,not_detected,not_detected
4,HTMCP-03-06-02006,HPV_45,integrated,not_reviewed,HTMCP_129,na,na,na,A37704,YES,12_46339702;12_46337036;12_46337035;12_4632994...,SCAF11;SCAF11;SCAF11;SCAF11;SCAF11;ARID2,integrated,integrated,integrated


In [34]:
df['genes'] = df['DNA_human-gene(s)'].apply(lambda x: x.split(';')) + df['RNA_human-gene(s)'].apply(lambda x: x.split(';'))

In [35]:
df

,patient,virus,reviewed_results,manual_review_notes,source,DNA_Integration,DNA_human-breakpoint,DNA_human-gene(s),library,RNA_Integration,RNA_human-breakpoint,RNA_human-gene(s),124_results,107_results,101_results,genes
0,HTMCP-03-06-02001,HPV_16,integrated,not_reviewed,HTMCP_124,na,na,na,A37700,YES,3_189542968;3_189542598;3_189542597;3_18952631...,TP63,integrated,integrated,integrated,"[na, TP63]"
1,HTMCP-03-06-02002,HPV_52,integrated,not_reviewed,HTMCP_125,na,na,na,A37701,YES,22_43168251;22_43165135;22_43163674;22_4316152...,intergenic,integrated,integrated,integrated,"[na, intergenic]"
2,HTMCP-03-06-02003,HPV_16,integrated,not_reviewed,HTMCP_126,NO,-,-,A37702,YES,11_60935033;11_60882522,intergenic;CD5,integrated,integrated,integrated,"[-, intergenic, CD5]"
3,HTMCP-03-06-02006,HPV_18,not_detected,reviewed,HTMCP_129,na,na,na,A37704,NO,-,-,NOT_INTEGRATED,not_detected,not_detected,"[na, -]"
4,HTMCP-03-06-02006,HPV_45,integrated,not_reviewed,HTMCP_129,na,na,na,A37704,YES,12_46339702;12_46337036;12_46337035;12_4632994...,SCAF11;SCAF11;SCAF11;SCAF11;SCAF11;ARID2,integrated,integrated,integrated,"[na, SCAF11, SCAF11, SCAF11, SCAF11, SCAF11, A..."
5,HTMCP-03-06-02007,no_HPV,not_detected,not_reviewed,HTMCP_130,na,na,na,A37705,na,na,na,not_detected,not_detected,not_detected,"[na, na]"
6,HTMCP-03-06-02008,HPV_16,unintegrated,not_reviewed,HTMCP_131,na,na,na,A37706,NO,-,-,unintegrated,unintegrated,unintegrated,"[na, -]"
7,HTMCP-03-06-02008,HPV_9,integrated,not_reviewed,HTMCP_131,na,na,na,A37706,YES,9_115288280,KIAA1958,integrated,integrated,integrated,"[na, KIAA1958]"
8,HTMCP-03-06-02012,HPV_16,integrated,not_reviewed,HTMCP_133,na,na,na,A37707,YES,14_38057621;14_38057528,intergenic,integrated,integrated,integrated,"[na, intergenic]"
9,HTMCP-03-06-02012,HPV_18,unintegrated,not_reviewed,HTMCP_133,na,na,na,A37707,NO,-,-,unintegrated,unintegrated,unintegrated,"[na, -]"


In [31]:
df1['DNA_human-gene(s)'] + df1['RNA_human-gene(s)']

0                                             [na, TP63]
1                                       [na, intergenic]
2                                   [-, intergenic, CD5]
3                                                [na, -]
4      [na, SCAF11, SCAF11, SCAF11, SCAF11, SCAF11, A...
5                                               [na, na]
6                                                [na, -]
7                                         [na, KIAA1958]
8                                       [na, intergenic]
9                                                [na, -]
10                                                [-, -]
11                                               [na, -]
12     [intergenic, intergenic, LOC440434, TBC1D3,LOC...
13                                            [na, FHIT]
14                                               [na, -]
15                                      [na, intergenic]
16                                              [na, na]
17                             

In [5]:
lst = list((df1['DNA_human-gene(s)'] + df1['RNA_human-gene(s)']))


In [6]:
lst

[['na', 'TP63'],
 ['na', 'intergenic'],
 ['-', 'intergenic', 'CD5'],
 ['na', '-'],
 ['na', 'SCAF11', 'SCAF11', 'SCAF11', 'SCAF11', 'SCAF11', 'ARID2'],
 ['na', 'na'],
 ['na', '-'],
 ['na', 'KIAA1958'],
 ['na', 'intergenic'],
 ['na', '-'],
 ['-', '-'],
 ['na', '-'],
 ['intergenic', 'intergenic', 'LOC440434', 'TBC1D3,LOC440434', 'TBC1D3'],
 ['na', 'FHIT'],
 ['na', '-'],
 ['na', 'intergenic'],
 ['na', 'na'],
 ['-', '-'],
 ['-', 'TMEM242'],
 ['na', 'intergenic'],
 ['CDC42SE1',
  'MLLT11',
  'MLLT11',
  'MLLT11',
  'MLLT11',
  'CDC42SE1',
  'CDC42SE1',
  'CDC42SE1',
  'CDC42SE1',
  'CDC42SE1',
  'PRUNE',
  'PRUNE',
  'PRUNE',
  'PRUNE'],
 ['intergenic',
  'STK11',
  'STK11',
  'STK11',
  'STK11',
  'STK11',
  'intergenic',
  'intergenic',
  'intergenic',
  'intergenic',
  'intergenic',
  'intergenic',
  'intergenic',
  'intergenic',
  'intergenic',
  'intergenic'],
 ['na', '-'],
 ['na',
  'BC037497',
  'KLF12',
  'KLF12',
  'KLF12',
  'KLF12',
  'KLF12',
  'KLF12',
  'KLF12',
  'KLF12',
  'K

In [7]:
# after flattening there are still genes separated by , becasue they are in orignal integration summary file, maybe overlap genes
flat_list = [item for sublist in lst for item in sublist]


In [8]:
st = set(flat_list)

In [9]:
new_lst = [s.split(',') for s in st]

In [19]:
genes = list(set([item for sublist in new_lst for item in sublist]))

In [20]:
len(genes)

215

In [21]:
def unique(a):
    """ return the list with duplicate elements removed """
    return list(set(a))

def intersect(a, b):
    """ return the intersection of two lists """
    return list(set(a) & set(b))

def union(a, b):
    """ return the union of two lists """
    return list(set(a) | set(b))

def exclusive(a, b):
    """ return items either exclusive in a or b """
    return list(set(a) ^ set(b))

def unique2a(a, b):
    """ return items only in a, but not in b """
    return list(set(a) & set(set(a) ^ set(b)))

def write(lst, of):
    with open(of, "w") as output:
        writer = csv.writer(output, lineterminator='\n')
        for val in lst:
            writer.writerow([val]) 

In [24]:
f = '/home/szong/projects/resource/oncogenes.and.tumor.suppressor.genes.txt'
df = pd.read_csv(f, header=None)
df.head(2)

,0
0,ABI1
1,ABI2


In [25]:
oncogenes = df[0].tolist()

In [36]:
for i in intersect(genes, oncogenes):
    print(i)

ERBB4
TERT
SOX7
BCL11B
WWOX
ARID2
DUX4
LRP1B
FHIT
JUP
TMPRSS11A
BMPR1A
CRTC3
TP63
ERBB2
STK11
HINT1
PINX1
EML4
MYC
NR4A1


In [38]:
', '.join(intersect(genes, oncogenes))

'ERBB4, TERT, SOX7, BCL11B, WWOX, ARID2, DUX4, LRP1B, FHIT, JUP, TMPRSS11A, BMPR1A, CRTC3, TP63, ERBB2, STK11, HINT1, PINX1, EML4, MYC, NR4A1'

In [ ]:
# ncbi entrez gene ids
2066 83595 64919 51741 196528 100288687 53353 2272 339967 657 64784 8626 3094 54984 27436 2064 6794 4609 7015 3728 3164

In [39]:
', '.join([2066 83595 64919 51741 196528 100288687 53353 2272 339967 657 64784 8626 3094 54984 27436 2064 6794 4609 7015 3728 3164])

SyntaxError: invalid syntax (<ipython-input-39-ae7b0e25e688>, line 1)

In [61]:
lst = ['ERBB4', 'TERT', 'SOX7', 'BCL11B', 'WWOX', 'ARID2', 'DUX4', 'LRP1B', 'FHIT', 'JUP', 'TMPRSS11A', 'BMPR1A', 'CRTC3', 'TP63', 'ERBB2', 'STK11', 'HINT1', 'PINX1', 'EML4', 'MYC', 'NR4A1']

In [ ]:
df[df.]

In [97]:
f = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/124_patients/124_patients_integration_details_merged_comparison_oncogenes.txt'
df = pd.read_csv(f)
df.head(2)

,patient,virus,integration_reported,107_results,101_results,integration_detected,source,DNA_Integration,DNA_human-breakpoint,DNA_human-gene(s),library,RNA_Integration,RNA_human-breakpoint,RNA_human-gene(s)
0,HTMCP-03-06-02411,HPV_16,integrated,NaN,NaN,BOTH_INTEGRATED,HTMCP_695,YES,X_53154732;X_154741079;X_154737381;X_11575288;...,intergenic;TMLHE;TMLHE;ARHGAP6;RNA5-8S5;interg...,A78477,YES,2_212989633;2_212870843;2_212868974;2_21283335...,ERBB4;ERBB4;ERBB4;ERBB4;ERBB4;CARD14;CARD14;CA...
1,HTMCP-03-06-02437,HPV_16,integrated,NaN,NaN,DNA_INTEGRATED,HTMCP_713,YES,5_981231;5_979816;5_1295420;5_1271287;4_49635864,intergenic;intergenic;intergenic;TERT;intergenic,A78495,NO,-,-


In [98]:
df['genes'] = df['DNA_human-gene(s)'].apply(lambda x: x.split(';')) + df['RNA_human-gene(s)'].apply(lambda x: x.split(';'))

In [99]:
df.head(2)

,patient,virus,integration_reported,107_results,101_results,integration_detected,source,DNA_Integration,DNA_human-breakpoint,DNA_human-gene(s),library,RNA_Integration,RNA_human-breakpoint,RNA_human-gene(s),genes
0,HTMCP-03-06-02411,HPV_16,integrated,NaN,NaN,BOTH_INTEGRATED,HTMCP_695,YES,X_53154732;X_154741079;X_154737381;X_11575288;...,intergenic;TMLHE;TMLHE;ARHGAP6;RNA5-8S5;interg...,A78477,YES,2_212989633;2_212870843;2_212868974;2_21283335...,ERBB4;ERBB4;ERBB4;ERBB4;ERBB4;CARD14;CARD14;CA...,"[intergenic, TMLHE, TMLHE, ARHGAP6, RNA5-8S5, ..."
1,HTMCP-03-06-02437,HPV_16,integrated,NaN,NaN,DNA_INTEGRATED,HTMCP_713,YES,5_981231;5_979816;5_1295420;5_1271287;4_49635864,intergenic;intergenic;intergenic;TERT;intergenic,A78495,NO,-,-,"[intergenic, intergenic, intergenic, TERT, int..."


In [100]:
df.shape

(25, 15)

In [101]:
# df.genes

In [102]:
df.genes = df.genes.apply(lambda x: ','.join(set(x) -{'intergenic'} - {'na'} -{'-'}))

In [103]:
df.head(2)

,patient,virus,integration_reported,107_results,101_results,integration_detected,source,DNA_Integration,DNA_human-breakpoint,DNA_human-gene(s),library,RNA_Integration,RNA_human-breakpoint,RNA_human-gene(s),genes
0,HTMCP-03-06-02411,HPV_16,integrated,NaN,NaN,BOTH_INTEGRATED,HTMCP_695,YES,X_53154732;X_154741079;X_154737381;X_11575288;...,intergenic;TMLHE;TMLHE;ARHGAP6;RNA5-8S5;interg...,A78477,YES,2_212989633;2_212870843;2_212868974;2_21283335...,ERBB4;ERBB4;ERBB4;ERBB4;ERBB4;CARD14;CARD14;CA...,"ARHGAP6,LINC00535,ERBB4,CBX8,GTF2F2,TMLHE,STK3..."
1,HTMCP-03-06-02437,HPV_16,integrated,NaN,NaN,DNA_INTEGRATED,HTMCP_713,YES,5_981231;5_979816;5_1295420;5_1271287;4_49635864,intergenic;intergenic;intergenic;TERT;intergenic,A78495,NO,-,-,TERT


In [104]:
df = df[['patient', 'virus', 'genes']]

In [105]:
df['p_v'] = df.patient + '_' +df.virus

In [106]:
df.head(2)

,patient,virus,genes,p_v
0,HTMCP-03-06-02411,HPV_16,"ARHGAP6,LINC00535,ERBB4,CBX8,GTF2F2,TMLHE,STK3...",HTMCP-03-06-02411_HPV_16
1,HTMCP-03-06-02437,HPV_16,TERT,HTMCP-03-06-02437_HPV_16


In [107]:
ndf = pd.concat([pd.Series(row['p_v'], row['genes'].split(',')) for idx, row in df.iterrows()]).reset_index()
ndf.head(2)

,index,0
0,ARHGAP6,HTMCP-03-06-02411_HPV_16
1,LINC00535,HTMCP-03-06-02411_HPV_16


In [108]:
ndf.shape

(136, 2)

In [113]:
of = '/projects/trans_scratch/validations/workspace/szong/Cervical/integration/124_patients/124_patients_integration_details_merged_comparison_oncogenes_patients.txt'
odf = ndf[ndf['index'].isin(lst)].groupby('index').apply(lambda x: ', '.join(x[0].unique())).to_frame()
odf.to_csv(of, sep='\t')

In [112]:
ndf[ndf['index'].isin(lst)].groupby('index').apply(lambda x: ', '.join(x[0].unique()))

index
ARID2                                 HTMCP-03-06-02006_HPV_45
BCL11B                                HTMCP-03-06-02202_HPV_16
BMPR1A                                HTMCP-03-06-02175_HPV_58
CRTC3                                 HTMCP-03-06-02092_HPV_45
DUX4                                  HTMCP-03-06-02427_HPV_18
EML4                                  HTMCP-03-06-02326_HPV_45
ERBB2                                 HTMCP-03-06-02109_HPV_16
ERBB4                                 HTMCP-03-06-02411_HPV_16
FHIT         HTMCP-03-06-02013_HPV_16, HTMCP-03-06-02125_HP...
HINT1                                 HTMCP-03-06-02194_HPV_16
JUP                                   HTMCP-03-06-02414_HPV_18
LRP1B                                 HTMCP-03-06-02063_HPV_16
MYC                                   HTMCP-03-06-02149_HPV_82
NR4A1                                 HTMCP-03-06-02109_HPV_16
PINX1                                 HTMCP-03-06-02322_HPV_16
SOX7                                  HTMCP-03-06

In [68]:
for g, d in ndf[ndf['index'].isin(lst)].groupby('index'):
    print(g)
    print(d)

ARID2
    index                         0
36  ARID2  HTMCP-03-06-02006_HPV_45
BCL11B
     index                         0
26  BCL11B  HTMCP-03-06-02202_HPV_16
CRTC3
    index                         0
75  CRTC3  HTMCP-03-06-02092_HPV_45
FHIT
   index                         0
50  FHIT  HTMCP-03-06-02013_HPV_16
51  FHIT  HTMCP-03-06-02125_HPV_16
HINT1
    index                         0
99  HINT1  HTMCP-03-06-02194_HPV_16
JUP
   index                         0
52   JUP  HTMCP-03-06-02414_HPV_18
LRP1B
    index                         0
48  LRP1B  HTMCP-03-06-02063_HPV_16
MYC
    index                         0
119   MYC  HTMCP-03-06-02149_HPV_82
NR4A1
     index                         0
83   NR4A1  HTMCP-03-06-02109_HPV_16
121  NR4A1  HTMCP-03-06-02109_HPV_16
SOX7
    index                         0
13   SOX7  HTMCP-03-06-02322_HPV_16
20   SOX7  HTMCP-03-06-02322_HPV_16
102  SOX7  HTMCP-03-06-02322_HPV_16
109  SOX7  HTMCP-03-06-02322_HPV_16
STK11
    index                         0
98 

In [26]:
a = pd.DataFrame([{'var1': 'a,b,c', 'var2': 1},
               {'var1': 'd,e,f', 'var2': 2}])

In [28]:
a

,var1,var2
0,"a,b,c",1
1,"d,e,f",2


In [30]:
pd.concat([pd.Series(row['var2'], row['var1'].split(','))              
                    for _, row in a.iterrows()]).reset_index()

,index,0
0,a,1
1,b,1
2,c,1
3,d,2
4,e,2
5,f,2


In [32]:
[pd.Series(row['var2'], row['var1'].split(',')) for _, row in a.iterrows()]

[a    1
 b    1
 c    1
 dtype: int64, d    2
 e    2
 f    2
 dtype: int64]